<a href="https://colab.research.google.com/github/hicbcb/AI_course/blob/main/week10(%E6%95%99%E5%AD%B8%E7%94%A8)_GAN%E5%9C%96%E5%83%8F%E5%89%B5%E4%BD%9CMNIST(%E5%84%AA%E5%8C%96%E9%81%8E).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import imageio

In [ ]:
# 超參數設置
latent_dim = 100
img_shape = (1, 28, 28)  # MNIST 圖像大小

batch_size = 128
num_epochs = 100
sample_interval = 400  # 每隔多少個批次生成一次圖像
max_norm = 1.0  # 增加梯度裁剪的閾值

In [ ]:
# 加載 MNIST 數據集
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # 調整圖像大小為 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17188768.20it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 527406.13it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3806597.69it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2823555.47it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# 定義生成器
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), self.latent_dim, 1, 1)
        img = self.model(z)
        return img

In [ ]:
# 定義判別器
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

In [ ]:
class GradientEarlyStopping:
    def __init__(self, patience=5, threshold=1.0):
        self.patience = patience
        self.threshold = threshold
        self.counter = 0
        self.should_stop = False

    def __call__(self, grad_norm):
        if grad_norm > self.threshold:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        else:
            self.counter = 0

    def reset(self):
        self.counter = 0
        self.should_stop = False

In [ ]:
# 創建生成器和判別器
generator = Generator(latent_dim)
discriminator = Discriminator()

In [ ]:
# 調整生成器和判別器的學習率
lr_g = 0.0002  # 生成器的學習率
lr_d = 0.0002  # 判別器的學習率

In [ ]:
# 定義優化器
optimizer_G = optim.Adam(generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))

In [ ]:
# 定義損失函數
#criterion = nn.BCELoss()

In [ ]:
# 用於保存生成圖像的列表
img_list = []

In [ ]:
# 創建 GradientEarlyStopping 實例
# early_stopping = GradientEarlyStopping(patience=10, threshold=2.0)

# 訓練循環
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(train_loader):

        # 訓練判別器
        z = torch.randn(imgs.size(0), latent_dim)
        fake_imgs = generator(z)

        real_validity = discriminator(imgs)
        fake_validity = discriminator(fake_imgs.detach())

        # 計算判別器損失
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity)

        # 計算梯度懲罰項
        alpha = torch.rand(imgs.size(0), 1, 1, 1)
        interpolates = (alpha * imgs + (1 - alpha) * fake_imgs).requires_grad_(True)
        d_interpolates = discriminator(interpolates)
        fake = torch.ones_like(d_interpolates, requires_grad=False)
        gradients = torch.autograd.grad(
            outputs=d_interpolates,
            inputs=interpolates,
            grad_outputs=fake,
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )[0]
        gradients = gradients.view(gradients.size(0), -1)
        gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
        d_loss += 10 * gradient_penalty  # 添加梯度懲罰項到判別器損失中

        optimizer_D.zero_grad()
        d_loss.backward()

        # 對判別器的梯度進行裁剪
        torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=max_norm)

        optimizer_D.step()

        # 訓練生成器
        z = torch.randn(imgs.size(0), latent_dim)
        fake_imgs = generator(z)
        fake_validity = discriminator(fake_imgs)

        # 計算生成器損失
        g_loss = -torch.mean(fake_validity)

        optimizer_G.zero_grad()
        g_loss.backward()

        # 對生成器的梯度進行裁剪
        torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=max_norm)

        optimizer_G.step()

        # 顯示圖
        batches_done = epoch * len(train_loader) + i
        if batches_done % sample_interval == 0:
            # 生成一批圖像並添加到列表中
            z = torch.randn(25, latent_dim)
            labels = torch.randint(0, 10, (25,))  # 隨機生成25個數字標籤
            gen_imgs = generator(z)
            #print("Generator output shape:", gen_imgs.shape)
            #gen_imgs = gen_imgs.view(25, 1, 28, 28)


            # 將數字標籤添加到生成的圖像上
            fig, axs = plt.subplots(5, 5, figsize=(5, 5))
            for j in range(5):
                for k in range(5):
                    idx = j * 5 + k
                    axs[j, k].imshow(gen_imgs[idx].detach().cpu().numpy().squeeze(), cmap='gray')
                    axs[j, k].set_title(str(labels[idx].item()), fontsize=12)
                    axs[j, k].axis('off')
            plt.tight_layout()
            #plt.show()


            # 將圖像轉換為 numpy 數組並添加到列表中
            fig.canvas.draw()
            img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            # 將生成的圖像添加到列表中
            img_list.append(img)

            plt.close(fig)

        # 計算生成器和判別器的梯度範數
        grad_norm_g = torch.norm(torch.cat([p.grad.view(-1) for p in generator.parameters()]))
        grad_norm_d = torch.norm(torch.cat([p.grad.view(-1) for p in discriminator.parameters()]))
        '''
        # 檢查梯度範數是否超過閾值
        early_stopping(max(grad_norm_g, grad_norm_d))

        if early_stopping.should_stop:
            print("Training stopped due to gradient explosion.")
            break

    if early_stopping.should_stop:
        break
        '''

    print(f"Epoch [{epoch+1}/{num_epochs}] D_loss: {d_loss.item():.4f} G_loss: {g_loss.item():.4f}")

    # 定期保存模型權重
    if (epoch + 1) % 50 == 0:
        torch.save(generator.state_dict(), f'generator_epoch_{epoch+1}.pth')
        torch.save(discriminator.state_dict(), f'discriminator_epoch_{epoch+1}.pth')

Epoch [1/100] D_loss: -0.0214 G_loss: -0.0081
Epoch [2/100] D_loss: -0.9824 G_loss: -0.0017
Epoch [3/100] D_loss: 0.0440 G_loss: -0.4304
Epoch [4/100] D_loss: -0.5793 G_loss: -0.0805


KeyboardInterrupt: 

In [ ]:
# 使用 imageio 創建動圖
with imageio.get_writer('gan_animation.gif', mode='I') as writer:
    for img in img_list:
        writer.append_data(img)